## 1. Instalación de Librerías 🛠️

En este paso instalamos las bibliotecas necesarias para el proyecto:
* **`fastbook`**: Incluye la librería `fastai` y herramientas útiles para el aprendizaje profundo.
* **`duckduckgo-search`**: Nos permitirá buscar y descargar imágenes de internet automáticamente para crear nuestro propio dataset.

In [1]:
!pip install fastbook duckduckgo-search

## 2. Importación y Verificación ✅

Aquí importamos las herramientas al entorno de trabajo actual:
* Importamos **`fastbook`** para tener acceso a sus funciones.
* Traemos la clase **`DDGS`**, que es el motor que usaremos para buscar imágenes en DuckDuckGo.
* Ejecutamos un mensaje de prueba para confirmar que no hubo errores en la instalación y que estamos listos para continuar.

In [2]:
import fastbook
from duckduckgo_search import DDGS

print("¡Todo funciona correctamente! Podemos seguir.")

¡Todo funciona correctamente! Podemos seguir.


## 3. Construcción del Dataset: Búsqueda y Descarga 📂

Este es el bloque principal del script. Aquí automatizamos la creación de nuestro conjunto de datos (dataset) para no tener que descargar fotos manualmente:

1.  **Función `search_images`**: Creamos una herramienta que usa `DuckDuckGo` para buscar imágenes y extraer sus URLs (direcciones de internet).
2.  **Categorías**: Definimos las clases que queremos que la IA aprenda: `avion`, `auto` y `barco`.
3.  **Bucle de Descarga**: El código recorre cada categoría una por una:
    * Crea una carpeta específica en tu computadora.
    * Busca las fotos en internet.
    * Usa la función `download_images` de **FastAI** para guardar los archivos automáticamente.
4.  **Pausa de Seguridad**: Incluimos `time.sleep(5)` para esperar 5 segundos entre búsquedas y evitar que el buscador nos bloquee por ir demasiado rápido.

In [3]:
# --- CÓDIGO---
from fastbook import * # <--- Esta línea es la clave que nos faltaba
from duckduckgo_search import DDGS
import time

def search_images(term, max_images=100):
    print(f"Buscando imágenes de: {term}")
    with DDGS() as ddgs:
        results = ddgs.images(keywords=term, max_results=max_images)
        urls = [r['image'] for r in results]
        return urls

# 1. Definimos las clases y la ruta
searches = 'avion', 'auto', 'barco'
path = Path('../data/images')

# 2. Bucle de descarga
for search_term in searches:
    dest = (path/search_term)
    dest.mkdir(exist_ok=True, parents=True)
    
    # Buscamos urls
    print(f"--- Procesando: {search_term} ---")
    # Usamos try/except por si acaso la búsqueda falla momentáneamente
    try:
        urls = search_images(f"{search_term} photo")
        
        # Descargamos
        if urls:
            print(f"Descargando imágenes en {dest}...")
            download_images(dest, urls=urls)
        else:
            print(f"No se encontraron imágenes para {search_term}")
            
    except Exception as e:
        print(f"Hubo un pequeño error buscando {search_term}: {e}")
    
    # Pausa de cortesía
    time.sleep(5)
    
print("\n¡Descarga terminada! Ahora revisa tus carpetas.")

--- Procesando: avion ---
Buscando imágenes de: avion photo


C:\Users\alexc\AppData\Local\Temp\ipykernel_33900\3230507251.py:8: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Descargando imágenes en ..\data\images\avion...
--- Procesando: auto ---
Buscando imágenes de: auto photo


C:\Users\alexc\AppData\Local\Temp\ipykernel_33900\3230507251.py:8: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Descargando imágenes en ..\data\images\auto...
--- Procesando: barco ---
Buscando imágenes de: barco photo


C:\Users\alexc\AppData\Local\Temp\ipykernel_33900\3230507251.py:8: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Hubo un pequeño error buscando barco: https://duckduckgo.com/i.js?o=json&q=barco+photo&l=us-en&vqd=4-264177483899785966346133305997828723666&p=1&f=%2C%2C%2C%2C%2C 403 Ratelimit

¡Descarga terminada! Ahora revisa tus carpetas.


## 4. Código de Rescate: Solución Específica para Barcos 🚤⚓

Este bloque es un script de **corrección manual**. Lo usamos si notamos que la categoría "barco" falló en el bucle anterior o quedó vacía. Sus características son:

1.  **Cambio de Estrategia**: En lugar de buscar "barco", buscamos **"boat photo"** (en inglés). Cambiar el idioma ayuda a obtener resultados frescos y a veces evita bloqueos temporales del buscador.
2.  **Ruta Forzada**: Nos aseguramos explícitamente de que las imágenes vayan a la carpeta `../data/images/barco`, sin importar qué palabra usemos para buscar.
3.  **Función Aislada**: Definimos una función `search_images_rescue` independiente para no interferir con el código principal y probar solo esta descarga.

In [5]:
# CÓDIGO DE RESCATE: SOLO PARA BARCOS
from fastbook import *
from duckduckgo_search import DDGS
import time

# Definimos la ruta de nuevo por si acaso
path = Path('../data/images')
dest = (path/'barco') # Guardamos en la carpeta 'barco' aunque busquemos 'boat'
dest.mkdir(exist_ok=True, parents=True)

print("--- Intentando descargar barcos (usando búsqueda 'boat')... ---")

def search_images_rescue(term, max_images=100):
    with DDGS() as ddgs:
        results = ddgs.images(keywords=term, max_results=max_images)
        urls = [r['image'] for r in results]
        return urls

try:
    # Buscamos 'boat photo' en vez de 'barco' para variar
    urls = search_images_rescue("boat photo")
    
    if urls:
        print(f"¡Éxito! Encontré {len(urls)} imágenes. Descargando...")
        download_images(dest, urls=urls)
        print("Descarga de barcos terminada.")
    else:
        print("No encontré imágenes. Intenta esperar 5 minutos más.")

except Exception as e:
    print(f"Sigue dando error: {e}")

--- Intentando descargar barcos (usando búsqueda 'boat')... ---


C:\Users\alexc\AppData\Local\Temp\ipykernel_33900\2273519863.py:14: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


¡Éxito! Encontré 100 imágenes. Descargando...
Descarga de barcos terminada.


## 5. Generación del Archivo de Etiquetas (CSV) 📝📊

Una vez descargadas las imágenes, necesitamos organizar la información en un formato estructurado. En este paso:

1.  **Escaneo de Carpetas**: El código recorre las carpetas `avion`, `auto` y `barco` que creamos anteriormente.
2.  **Asociación de Etiquetas**: Para cada imagen encontrada, el código asume que la etiqueta correcta es el nombre de la carpeta donde está guardada (ej: si está en la carpeta "avion", su etiqueta es "avion").
3.  **Creación de la Tabla**: Usamos la librería **Pandas** para crear un "DataFrame" (una tabla tipo Excel) con dos columnas:
    * `fname`: La ruta relativa de la imagen (ej: `avion/001.jpg`).
    * `labels`: La categoría a la que pertenece.
4.  **Guardado**: Exportamos esta tabla como un archivo **`labels.csv`** dentro de la carpeta `data`. Este archivo será el "mapa" que guiará el entrenamiento de nuestro modelo.

In [6]:
import pandas as pd
import os

# Ruta donde están tus imágenes
path = Path('../data/images')

# Listas para guardar la información
nombres_archivos = []
etiquetas = []

# Recorremos cada carpeta (avion, auto, barco)
classes = ['avion', 'auto', 'barco']

for clase in classes:
    folder_path = path/clase
    
    # Verificamos que la carpeta exista
    if folder_path.exists():
        # Listamos los archivos dentro
        files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
        
        for file in files:
            # Guardamos el nombre del archivo (ej: avion/001.jpg)
            nombres_archivos.append(f"{clase}/{file}")
            # Guardamos la etiqueta
            etiquetas.append(clase)
            
    else:
        print(f"Cuidado: La carpeta {clase} no existe o está mal escrita.")

# Creamos el DataFrame (la tabla)
df = pd.DataFrame({
    'fname': nombres_archivos,
    'labels': etiquetas
})

# Guardamos el archivo CSV en la carpeta data
csv_path = Path('../data/labels.csv')
df.to_csv(csv_path, index=False)

print(f"¡Éxito! Archivo creado en: {csv_path}")
print(f"Total de imágenes encontradas: {len(df)}")
print("\nPrimeras 5 filas del archivo:")
print(df.head())

¡Éxito! Archivo creado en: ..\data\labels.csv
Total de imágenes encontradas: 281

Primeras 5 filas del archivo:
                                            fname labels
0  avion/022ae613-75a9-4f95-8745-1e656e03ba56.jpg  avion
1  avion/03783d6a-63eb-41f8-a52f-767222bab38a.jpg  avion
2  avion/094e97e1-17e7-455b-9b67-187444bf156f.jpg  avion
3  avion/0c9c68a3-94dc-4cdc-aeb1-d3b86918486f.jpg  avion
4  avion/0cc51a5c-9e62-48be-a331-a814b279839f.jpg  avion
